In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.utils.data as Data
import torch.optim as optim

In [2]:
char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz'] # ['a', 'b', 'c',...]
word2idx = {n: i for i, n in enumerate(char_arr)}
idx2word = {i: w for i, w in enumerate(char_arr)}
n_class = len(word2idx) # number of class(=number of vocab)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
n_step = len(seq_data[0]) - 1 # (=3)
n_hidden = 128


In [3]:
def make_data(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word2idx[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word2idx[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)
input_batch, target_batch = make_data(seq_data)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 3, True)

In [4]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM,self).__init__()
        self.lstm = nn.LSTM(input_size=n_class,hidden_size=n_hidden,num_layers=1,batch_first=True)
        self.fc = nn.Linear(n_hidden,n_class)
    def forward(self,X):
        batch_size =X.shape[0]
        hidden_state = torch.zeros(1,batch_size , n_hidden).cuda()   # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = torch.zeros(1,batch_size , n_hidden).cuda()    # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        output, (_, _)  = self.lstm(X,(hidden_state, cell_state))
        return self.fc(output[:,-1,:])

In [5]:
model = TextLSTM().cuda()
optimizer = optim.Adam(model.parameters(),lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [6]:
for epoch in range(5000):
    for x,y in loader:
        pred = model(x.cuda())
        loss = criterion(pred,y.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1) %1000 ==0:
           print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss),end='\r\n')

Epoch: 1000 cost = 0.000109
Epoch: 1000 cost = 0.000250
Epoch: 1000 cost = 0.000296
Epoch: 1000 cost = 0.000011
Epoch: 2000 cost = 0.000024
Epoch: 2000 cost = 0.000033
Epoch: 2000 cost = 0.000004
Epoch: 2000 cost = 0.000000
Epoch: 3000 cost = 0.000003
Epoch: 3000 cost = 0.000002
Epoch: 3000 cost = 0.000001
Epoch: 3000 cost = 0.000002
Epoch: 4000 cost = 0.000000
Epoch: 4000 cost = 0.000000
Epoch: 4000 cost = 0.000000
Epoch: 4000 cost = 0.000000
Epoch: 5000 cost = 0.000000
Epoch: 5000 cost = 0.000000
Epoch: 5000 cost = 0.000000
Epoch: 5000 cost = 0.000000


In [7]:
inputs = [sen[:3] for sen in seq_data]
predict = model(input_batch.cuda()).data.max(1, keepdim=True)[1]
print(inputs, '->', [idx2word[n.item()] for n in predict.cpu().data.squeeze()])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
